In [ ]:
# ============================================================================ #
# Copyright (c) 2022 - 2023 NVIDIA Corporation & Affiliates.                   #
# All rights reserved.                                                         #
#                                                                              #
# This source code and the accompanying materials are made available under     #
# the terms of the Apache License 2.0 which accompanies this distribution.     #
# ============================================================================ #

In [ ]:
import os
import pytest
import numpy as np

import cudaq

In [ ]:
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7474))]
molecule, data = cudaq.chemistry.create_molecular_hamiltonian(
        geometry, 'sto-3g', 1, 0)

exact_energy = molecule.to_matrix().minimal_eigenvalue()
print(type(molecule))
print("exact_energy = ", exact_energy)

# Get the number of fermions and orbitals / qubits
numElectrons = data.n_electrons
numQubits = 2 * data.n_orbitals

print(numQubits)

In [ ]:
# create the ansatz
kernel, thetas = cudaq.make_kernel(list)
qubits = kernel.qalloc(numQubits)

# hartree fock state
kernel.x(qubits[0])
kernel.x(qubits[1])

cudaq.kernels.uccsd(kernel, qubits, thetas, numElectrons, numQubits)

num_parameters = cudaq.kernels.uccsd_num_parameters(numElectrons, numQubits)

In [ ]:
# Run VQE
#optimizer = cudaq.optimizers.COBYLA()
optimizer = cudaq.optimizers.Adam()
gradient = cudaq.gradients.CentralDifference()
#optimizer.max_iterations = 1

#sys.exit()
energy, params = cudaq.vqe(kernel=kernel, spin_operator=molecule, optimizer=optimizer,
                gradient_strategy=gradient, parameter_count=num_parameters)
print(energy, params)
np.isclose(energy, -1.137, rtol=1e-3)